Reading in ADP data

In [10]:
import pandas as pd

# Reading in pff_receiving data

fantasy_adp_15_16 = pd.read_csv('../fantasy_data/fantasy_adp/fantasy_adp_15-16.csv')
fantasy_adp_16_17 = pd.read_csv('../fantasy_data/fantasy_adp/fantasy_adp_16-17.csv')
fantasy_adp_17_18 = pd.read_csv('../fantasy_data/fantasy_adp/fantasy_adp_17-18.csv')
fantasy_adp_18_19 = pd.read_csv('../fantasy_data/fantasy_adp/fantasy_adp_18-19.csv')
fantasy_adp_19_20 = pd.read_csv('../fantasy_data/fantasy_adp/fantasy_adp_19-20.csv')
fantasy_adp_20_21 = pd.read_csv('../fantasy_data/fantasy_adp/fantasy_adp_20-21.csv')
fantasy_adp_21_22 = pd.read_csv('../fantasy_data/fantasy_adp/fantasy_adp_21-22.csv')
fantasy_adp_22_23 = pd.read_csv('../fantasy_data/fantasy_adp/fantasy_adp_22-23.csv')
fantasy_adp_23_24 = pd.read_csv('../fantasy_data/fantasy_adp/fantasy_adp_23-24.csv')
fantasy_adp_24_25 = pd.read_csv('../fantasy_data/fantasy_adp/fantasy_adp_24-25.csv')

# Creating a season column to differentiate between seasons

fantasy_adp_15_16['Season'] = '2015-16'
fantasy_adp_16_17['Season'] = '2016-17'
fantasy_adp_17_18['Season'] = '2017-18'
fantasy_adp_18_19['Season'] = '2018-19'
fantasy_adp_19_20['Season'] = '2019-20'
fantasy_adp_20_21['Season'] = '2020-21'
fantasy_adp_21_22['Season'] = '2021-22'
fantasy_adp_22_23['Season'] = '2022-23'
fantasy_adp_23_24['Season'] = '2023-24'
fantasy_adp_24_25['Season'] = '2024-25'

fantasy_adp_15_16['position'] = fantasy_adp_24_25['POS'].str.slice(0,2)
fantasy_adp_16_17['position'] = fantasy_adp_24_25['POS'].str.slice(0,2)
fantasy_adp_17_18['position'] = fantasy_adp_24_25['POS'].str.slice(0,2)
fantasy_adp_18_19['position'] = fantasy_adp_24_25['POS'].str.slice(0,2)
fantasy_adp_19_20['position'] = fantasy_adp_24_25['POS'].str.slice(0,2)
fantasy_adp_20_21['position'] = fantasy_adp_24_25['POS'].str.slice(0,2)
fantasy_adp_21_22['position'] = fantasy_adp_24_25['POS'].str.slice(0,2)
fantasy_adp_22_23['position'] = fantasy_adp_24_25['POS'].str.slice(0,2)
fantasy_adp_23_24['position'] = fantasy_adp_24_25['POS'].str.slice(0,2)
fantasy_adp_24_25['position'] = fantasy_adp_24_25['POS'].str.slice(0,2)

# Combining all of the different season's data into one fantasy_adp dataset

fantasy_adp = pd.concat([fantasy_adp_15_16, fantasy_adp_16_17, fantasy_adp_17_18, fantasy_adp_18_19, fantasy_adp_19_20, fantasy_adp_20_21, fantasy_adp_21_22, fantasy_adp_22_23, fantasy_adp_23_24, fantasy_adp_24_25])




Setting Draft Settings

Naive draft pick strategy

Draft Analysis
ADP of drafted player order:



In [ ]:
import random
NUM_TEAMS = 10
ROSTER_SLOTS = {
    "QB": 1,
    "RB": 2,
    "WR": 2,
    "TE": 1,
    "FLEX": 2,  # RB/WR/TE
    "Bench": 6
}
TOTAL_ROUNDS = sum(ROSTER_SLOTS.values())

class Player:
    def __init__(self, player_row, round_taken, pick_position, projected_ppg=None):
        self.name = player_row['Player']
        self.position = player_row['position']
        self.team = player_row['Team']
        self.round_taken = round_taken
        self.pick_position = pick_position
        #self.pick_number = (round_taken - 1) * NUM_TEAMS + pick_position # Need to change how pick position is used in calculation dependent on snake order
        #self.projected_ppg = projected_ppg # Need to decide if I need to include this here or not

    def __repr__(self):
        return f"{self.name} ({self.position}) - ADP: {self.adp}"

class Team:
    def __init__(self, draft_position):
        self.roster = {"QB": [], "RB": [], "WR": [], "TE": [], "FLEX": [], "Bench": []}
        self.draft_position = draft_position


class Draft:
    def __init__(self, num_teams, roster_slots, fantasy_adp):
        # Instance attributes
        self.teams = {i: Team(i) for i in range(1, num_teams + 1)}
        self.current_round = 1
        self.current_pick_position = 1
        self.num_teams = num_teams
        self.roster_slots = roster_slots
        self.total_rounds = sum(roster_slots.values())
        self.available_players = fantasy_adp.copy()

    def advance_pick(self):
        if self.current_round % 2 == 1:
            # Odd round: left to right
            if self.current_pick_position >= self.num_teams:
                self.current_round += 1
            else:
                self.current_pick_position += 1
        else:
            # Even round: right to left
            if self.current_pick_position <= 1:
                self.current_round += 1
            else:
                self.current_pick_position -= 1

        if self.current_round > self.total_rounds:
            raise StopIteration("Draft is complete.")
        

    def pick_player(self, player):
        if len(self.teams[self.current_pick_position].roster[player.position]) < self.roster_slots[player.position]:
            self.teams[self.current_pick_position].roster[player.position].append(player)
        elif player.position in ["RB", "WR", "TE"] and len(self.teams[self.current_pick_position].roster["FLEX"]) < self.roster_slots["FLEX"]:
            # Add to FLEX if possible
            self.teams[self.current_pick_position].roster["FLEX"].append(player)
        else:
            self.teams[self.current_pick_position].roster["Bench"].append(player)

        self.available_players = self.available_players[self.available_players['Player'] != player.name]

        print(f"Round {self.current_round}, Position {self.current_pick_position}: Team {self.current_pick_position} picked {player.name} ({player.position})")


    def print_current_teams(self):
        for key, team in self.teams.items():
            print(f"Team {key} Roster:")
            for position, players in team.roster.items():
                if players:
                    print(f"  {position}: {[player.name for player in players]}")
            print()

        




Testing Draft class

In [24]:
import time
draft = Draft(10, ROSTER_SLOTS, fantasy_adp_24_25)


for i in range(60):
    rank = [0, 1, 2, 3, 4, 5]
    weights = [0.4, 0.3, 0.2, 0.1, 0.05, 0.05]
    chosen_rank = random.choices(rank, weights=weights, k=1)[0]
    player_choice = draft.available_players.iloc[chosen_rank]
    draft.pick_player(Player(player_choice, draft.current_round, draft.current_pick_position))
    draft.advance_pick()
    time.sleep(1)  # Adding a delay to simulate time between picks

draft.print_current_teams()

Round 1, Position 1: Team 1 picked Christian McCaffrey (RB)
Round 1, Position 2: Team 2 picked CeeDee Lamb (WR)
Round 1, Position 3: Team 3 picked Bijan Robinson (RB)
Round 1, Position 4: Team 4 picked Tyreek Hill (WR)
Round 1, Position 5: Team 5 picked A.J. Brown (WR)
Round 1, Position 6: Team 6 picked Amon-Ra St. Brown (WR)
Round 1, Position 7: Team 7 picked Ja'Marr Chase (WR)
Round 1, Position 8: Team 8 picked Justin Jefferson (WR)
Round 1, Position 9: Team 9 picked Breece Hall (RB)
Round 1, Position 10: Team 10 picked Kyren Williams (RB)
Round 2, Position 10: Team 10 picked Saquon Barkley (RB)
Round 2, Position 9: Team 9 picked Garrett Wilson (WR)
Round 2, Position 8: Team 8 picked Jahmyr Gibbs (RB)
Round 2, Position 7: Team 7 picked Travis Etienne Jr. (RB)
Round 2, Position 6: Team 6 picked Puka Nacua (WR)
Round 2, Position 5: Team 5 picked Marvin Harrison Jr. (WR)
Round 2, Position 4: Team 4 picked Jonathan Taylor (RB)
Round 2, Position 3: Team 3 picked Derrick Henry (RB)
Round 2

Creating Agents

In [ ]:
import random
from abc import ABC, abstractmethod


    # Create list of top players at needed positions
    # Make more likely to choose players at needed positions. Still give a chance to choose RB's and WR's if they are still needed.
    # More likekly to choose backup QB's and TE's if they didn't take one in the first 6ish rounds
    


class Agent(ABC):
    def __init__(self, name, draft_position):
        self.name = name
        self.draft_position = draft_position

    @abstractmethod
    def pick_player(self, available_players, team, roster_slots):
        """
        Decide which player to pick.
        Must be implemented by all subclasses.

        Parameters:
          - available_players: current pool of players to pick from
          - team: current Team object of the agent
          - roster_slots: draft roster configuration

        Returns:
          - Player object to pick
        """
        pass


class NaiveAgent:
    def __init__(self, name):
        self.name = name
        # You can store persistent info if needed, e.g. draft_position, preferences, etc.

    def pick_player(self, available_players, team, roster_slots):
        """
        Decide which player to pick based on:
          - available_players (e.g. a DataFrame or list of Player objects)
          - team (the Team object representing this agent's roster so far)
          - roster_slots (to know positional needs)
        Returns:
          - Player object to pick
        """
        # Implement your picking logic here
        pass

Running Draft Simulation

In [ ]:
def run_draft_simulation(num_teams, roster_slots, fantasy_adp, agents):
    """
    Run a full draft simulation.
    
    Parameters:
      - num_teams: int, number of teams drafting
      - roster_slots: dict, roster configuration
      - fantasy_adp: DataFrame or list of players available for drafting
      - agent_classes: dict mapping team position to Agent instances
                       e.g., {1: NaiveAgent(...), 2: ReinforcementLearningAgent(...), ...}
    """

    # Initialize the draft environment
    draft = Draft(num_teams, roster_slots, fantasy_adp)


    # Main draft loop
    done = False
    while not done:
        try:
            current_pick = draft.current_pick_position
            current_agent = agents[current_pick]

            # Let the agent pick a player given current available players and their team
            player_to_pick = current_agent.pick_player(
                draft.available_players, 
                draft.teams[current_pick], 
                draft.roster_slots
            )

            # Draft the player and advance
            draft.pick_player(player_to_pick)
            draft.advance_pick()

        except StopIteration:
            done = True

    # After draft, print rosters or return results
    draft.print_current_teams()